In [0]:
%run ./Setup

In [0]:
display(DA.paths.working_dir)


#select * from json.'{DA.paths.working_dir}/orders'
#dbfs:/Volumes/workspace/ops/volumes/orders
          
spark.sql(f'''
        select * 
        from json.`{DA.paths.working_dir}/orders`
    ''').display()



In [0]:
%sql

create or replace table default.orders_bronze
as
select
*,
current_timestamp() as processing_time,
_metadata.file_name as source_file
from read_files(
  "dbfs:/Volumes/workspace/ops/volumes" || "/orders",
  format => "json"
);

create or replace table default.orders_silver
as
select
order_id,
timestamp(ordertimestamp) as order_timestamp,
customer_id,
source_file
from default.orders_bronze;

create or replace view default.orders_by_date_vw
as
select
  date(order_timestamp) as order_date,
  count(*) total_daily_orders
from default.orders_silver
group by date(order_timestamp);


select * from default.orders_bronze;
select * from default.orders_silver;
select * from default.orders_by_date_vw;